### Import thư viện

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Đọc dữ liệu

In [2]:
data=pd.read_csv('data.csv')
print(data.shape)
data.sample(5)

(7445, 9)


,Date,Price,Open,High,Low,Vol.,Change %,Ticker Symbol,Bank Name
4136,09/27/2019,"15,769.3","15,769.3","15,881.1","15,769.3",3.10M,0.00%,CTG,Vietinbank
6856,10/07/2020,"5,039.3","4,999.9","5,098.3","4,999.9",632.42K,-0.77%,AGR,Agribank
2939,07/27/2018,"18,980.5","18,444.8","19,095.3","18,368.2",2.55M,2.90%,BID,BIDV
2635,10/15/2019,"31,685.2","31,379.1","31,800.0","31,379.1",794.63K,0.61%,BID,BIDV
7270,02/14/2019,"3,454.7","3,444.8","3,474.3","3,425.1",55.14K,0.29%,AGR,Agribank


### Xử lý dữ liệu

In [3]:
# Chuyển đổi cột 'Date' sang kiểu dữ liệu datetime
data['Date'] = pd.to_datetime(data['Date'])
# Chuyển đổi các cột số sang kiểu dữ liệu số
numeric_columns = ['Price', 'Open', 'High', 'Low', 'Change %']
for col in numeric_columns:
    data[col] = data[col].astype(str).str.replace(',', '').str.replace('%', '')
    data[col] = pd.to_numeric(data[col], errors='coerce')

def convert_volume(vol):
    if vol.endswith('M'):
        return float(vol[:-1]) * 1_000_000
    elif vol.endswith('K'):
        return float(vol[:-1]) * 1_000
    else:
        return float(vol)

if (data['Vol.'].dtype == 'O'):
    data['Vol.'] = data['Vol.'].apply(convert_volume)

# Sắp xếp dữ liệu theo cột 'Date' tăng dần
data = data.sort_values(by='Date')

# Kiểm tra loại dữ liệu của các cột
print(data.dtypes)

data.tail(5)

Date             datetime64[ns]
Price                   float64
Open                    float64
High                    float64
Low                     float64
Vol.                    float64
Change %                float64
Ticker Symbol            object
Bank Name                object
dtype: object


,Date,Price,Open,High,Low,Vol.,Change %,Ticker Symbol,Bank Name
1489,2024-05-20,49600.0,49450.0,50700.0,49400.0,2190000.0,0.51,BID,BIDV
5956,2024-05-20,20150.0,20100.0,20600.0,20100.0,1410000.0,0.25,AGR,Agribank
2978,2024-05-20,33350.0,33150.0,33900.0,33100.0,12150000.0,0.76,CTG,Vietinbank
4467,2024-05-20,49840.0,49750.0,50400.0,46750.0,9300000.0,0.18,TCB,Techcombank
0,2024-05-20,91900.0,92000.0,98200.0,88700.0,1040000.0,0.11,VCB,Vietcombank


### Câu hỏi: Có sự tương quan (có ý nghĩa về thống kê) giữa giá cổ phiếu và khối lượng giao dịch đối với từng ngân hàng; trong tổng thể hay không?

In [10]:
# Hàm tạo biểu đồ nến
def create_candlestick_chart(data, visible=False):
    return go.Candlestick(
        x=data['Date'],
        open=data['Open'],
        high=data['High'],
        low=data['Low'],
        close=data['Price'],
        visible=visible,
        showlegend=False,
        name=''
    )

# Hàm tạo biểu đồ khối lượng giao dịch
def create_volume_chart(data, visible=False):
    colors = ['red' if row['Price'] <= row['Open'] else 'green' for _, row in data.iterrows()]
    return go.Bar(
        x=data['Date'],
        y=data['Vol.'],
        marker=dict(color=colors),
        visible=visible,
        showlegend=False,
        name=''
    )

# Lấy danh sách các cổ phiếu
unique_tickers = data['Ticker Symbol'].unique()
print(unique_tickers)

# Tạo biểu đồ với 2 dòng và 1 cột
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.7, 0.3])

# Lấy dữ liệu theo từng cổ phiếu
data_grouped = [data[data['Ticker Symbol'] == ticker] for ticker in unique_tickers]
# print(data_grouped[0].head())

# Thêm traces cho cổ phiếu
for i in range(len(data_grouped)):
    fig.add_trace(create_candlestick_chart(data_grouped[i], visible=i == 0), row=1, col=1)
    fig.add_trace(create_volume_chart(data_grouped[i], visible=i == 0), row=2, col=1)


# Cập nhật layout với updatemenus
fig.update_layout(
    title=f'Khối lượng giao dịch và giá cổ phiếu {unique_tickers[0]}',
    xaxis={
        'rangeslider': {'visible': False},
        'title': 'Ngày',
        'titlefont': dict(  
            size=14, 
            color='black'  
        ),
    },
    yaxis_title='Giá cổ phiếu',
    yaxis2_title='Khối lượng giao dịch',
    updatemenus=[
        {
            'buttons': [
                {
                    'label': ticker,
                    'method': 'update',
                    'args': [
                        {'visible': [ticker == t for t in unique_tickers for _ in range(2)]},  # Update visibility for all candlestick traces
                        {'title': f'Khối lượng giao dịch và giá cổ phiếu - {ticker}'}  # Update title
                    ]
                }
                for ticker in unique_tickers
            ]
        }
    ]
)

# Hiển thị biểu đồ
fig.show()

['AGR' 'BID' 'CTG' 'VCB' 'TCB']


#### Nhận xét:
**Nhận xét chung:**
- Mối quan hệ tương quan giữa giá cổ phiếu và khối lượng giao dịch nhìn chung là tích cực. Khi giá cổ phiếu tăng, khối lượng giao dịch cũng có xu hướng tăng theo, và ngược lại. Điều này cho thấy rằng nhà đầu tư có xu hướng mua nhiều cổ phiếu hơn khi họ tin tưởng rằng giá cổ phiếu sẽ tăng, và bán nhiều cổ phiếu hơn khi họ tin tưởng rằng giá cổ phiếu sẽ giảm.
- Mức độ tương quan giữa giá cổ phiếu và khối lượng giao dịch có thể khác nhau tùy thuộc vào từng cổ phiếu. Một số cổ phiếu có mức độ tương quan cao hơn, trong khi một số cổ phiếu có mức độ tương quan thấp hơn.
**Nhận xét riêng từng loại cổ phiếu:**
1. AGR (Agribank): Mối quan hệ tương quan giữa giá cổ phiếu và khối lượng giao dịch của AGR là tương đối cao. Khi giá cổ phiếu AGR tăng, khối lượng giao dịch cũng có xu hướng tăng theo, và ngược lại.
2. BID (BIDV): Mối quan hệ tương quan giữa giá cổ phiếu và khối lượng giao dịch của BID là tương đối thấp.
3. CTG (Vietinbank): Mối quan hệ tương quan giữa giá cổ phiếu và khối lượng giao dịch của CTG là tương đối cao.
4. VCB (Vietcombank): Mối quan hệ tương quan giữa giá cổ phiếu và khối lượng giao dịch của VCB là tương đối thấp.
5. TCB (Techcombank): Mối quan hệ tương quan giữa giá cổ phiếu và khối lượng giao dịch của TCB là tương đối cao.

**Kết luận:**
- Mối quan hệ tương quan giữa giá cổ phiếu và khối lượng giao dịch nhìn chung là tích cực, nhưng mức độ tương quan có thể khác nhau tùy thuộc vào từng cổ phiếu.
- Có một số yếu tố khác có thể ảnh hưởng đến khối lượng giao dịch, ngoài giá cổ phiếu.

#### Kiểm định thống kê
Thực hiện kiểm định mô hình hồi quy tuyến tính OLS

In [8]:
# Thực hiện hồi quy tuyến tính cho từng ngân hàng
results = {}
for ticker in data['Ticker Symbol'].unique():
    bank_data = data[data['Ticker Symbol'] == ticker]
    model = smf.ols('Price ~ Q("Vol.")', data=bank_data).fit()
    results[ticker] = model.summary()

# In kết quả hồi quy cho từng ngân hàng
for ticker, summary in results.items():
    print(f'Results for {ticker}:')
    print(summary)
    print('\n')

Results for AGR:
                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.420
Model:                            OLS   Adj. R-squared:                  0.420
Method:                 Least Squares   F-statistic:                     1077.
Date:                Thu, 23 May 2024   Prob (F-statistic):          3.84e-178
Time:                        00:02:31   Log-Likelihood:                -14693.
No. Observations:                1487   AIC:                         2.939e+04
Df Residuals:                    1485   BIC:                         2.940e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   6182.4255    163.099   

#### Nhận xét:
1. **AGR (Agribank):**
- Giá trị `R-squared` và `Adj. R-squared` là 0.420, nghĩa là mô hình hồi quy giải thích được 42% biến động của giá cổ phiếu (Price) dựa trên khối lượng giao dịch (Vol.). Đây là một tỷ lệ giải thích khá, nhưng vẫn còn 58% biến động không được giải thích bởi mô hình này.
- Giá trị `F-statistic` là 1077 với mức ý nghĩa (`Prob`) rất nhỏ (3.84e-178), cho thấy mô hình có ý nghĩa thống kê, tức là khối lượng giao dịch có ảnh hưởng đáng kể đến giá cổ phiếu.
- Giá trị `p (P>|t|)`: Cả hai hệ số đều có giá trị p rất nhỏ (0.000), cho thấy chúng đều có ý nghĩa thống kê cao.
- Hệ số chặn (`Intercept`) là 6182.4255. Đây là giá trị trung bình của giá cổ phiếu khi khối lượng giao dịch bằng 0.
- Hệ số của khối lượng giao dịch là 0.0030. Điều này có nghĩa là khi khối lượng giao dịch tăng lên 1 đơn vị (sau khi chuyển đổi), giá cổ phiếu tăng trung bình 0.0030 đơn vị.
- Phương trình hồi quy: `Price=6182.4255 + 0.0030 * Vol.`
- **Kết luận**: Khối lượng giao dịch có ảnh hưởng đáng kể đến giá cổ phiếu của AGR. Hệ số dương cho thấy khi khối lượng giao dịch tăng, giá cổ phiếu cũng có xu hướng tăng.
2. **BID (BIDV):**
- Giá trị `R-squared` và `Adj. R-squared` là 0.022, nghĩa là mô hình hồi quy chỉ giải thích được 2.2% biến động của giá cổ phiếu (Price) dựa trên khối lượng giao dịch (Vol.). Đây là tỷ lệ giải thích rất thấp, cho thấy mô hình này không phải là một mô hình tốt để dự đoán giá cổ phiếu dựa trên khối lượng giao dịch cho BIDV.
- Giá trị `F-statistic` là 32.88 với mức ý nghĩa (`Prob`) rất nhỏ (1.18e-08), cho thấy mô hình có ý nghĩa thống kê, tức là khối lượng giao dịch có ảnh hưởng đáng kể đến giá cổ phiếu.
- Giá trị `p (P>|t|)`: Cả hai hệ số đều có giá trị p rất nhỏ (0.000), cho thấy chúng đều có ý nghĩa thống kê cao.
- Hệ số chặn (`Intercept`) là 36610. Đây là giá trị trung bình của giá cổ phiếu khi khối lượng giao dịch bằng 0.
- Hệ số của khối lượng giao dịch là -0.0008. Điều này có nghĩa là khi khối lượng giao dịch tăng lên 1 đơn vị (sau khi chuyển đổi), giá cổ phiếu giảm trung bình 0.0008 đơn vị.
- Phương trình hồi quy: `Price = 36610 - 0.0008 * Vol.`
- **Kết luận**: dù khối lượng giao dịch có ảnh hưởng đến giá cổ phiếu của BIDV nhưng mức độ giải thích của mô hình là rất thấp, và ảnh hưởng của khối lượng giao dịch là ngược chiều với giá cổ phiếu.
3. **CTG (VietinBank):**
- Giá trị `R-squared` và `Adj. R-squared` là 0.150, nghĩa là mô hình hồi quy giải thích được 15% biến động của giá cổ phiếu (Price) dựa trên khối lượng giao dịch (Vol.). Đây là một tỷ lệ giải thích trung bình, cho thấy mô hình này có thể dự đoán một phần giá cổ phiếu dựa trên khối lượng giao dịch.
- Giá trị `F-statistic` là 262.5 với mức ý nghĩa (`Prob`) rất nhỏ (1.72e-54), cho thấy mô hình có ý nghĩa thống kê, tức là khối lượng giao dịch có ảnh hưởng đáng kể đến giá cổ phiếu.
- Giá trị `p (P>|t|)`: Cả hai hệ số đều có giá trị p rất nhỏ (0.000), cho thấy chúng đều có ý nghĩa thống kê cao.
- Hệ số chặn (`Intercept`) là 20980. Đây là giá trị trung bình của giá cổ phiếu khi khối lượng giao dịch bằng 0.
- Hệ số của khối lượng giao dịch là 0.0005. Điều này có nghĩa là khi khối lượng giao dịch tăng lên 1 đơn vị (sau khi chuyển đổi), giá cổ phiếu tăng trung bình 0.0005 đơn vị.
- Phương trình hồi quy: `Price = 20980 + 0.0005 * Vol.`
- **Kết luận**: khối lượng giao dịch có ảnh hưởng đáng kể đến giá cổ phiếu của VietinBank, và mô hình hồi quy này có thể giải thích được 15% biến động của giá cổ phiếu dựa trên khối lượng giao dịch. Điều này cho thấy có một mức độ tương quan nhất định giữa hai biến này, nhưng vẫn còn 85% biến động không được giải thích bởi mô hình.
4. **VCB (Vietcombank):**
- Giá trị `R-squared` và `Adj. R-squared` là 0.042, nghĩa là mô hình hồi quy giải thích được 4.2% biến động của giá cổ phiếu (Price) dựa trên khối lượng giao dịch (Vol.). Đây là một tỷ lệ giải thích rất thấp, cho thấy mô hình này không thể dự đoán tốt giá cổ phiếu dựa trên khối lượng giao dịch.
- Giá trị `F-statistic` là 65.42 với mức ý nghĩa (`Prob`) rất nhỏ (1.24e-15), cho thấy mô hình có ý nghĩa thống kê, tức là khối lượng giao dịch có ảnh hưởng đáng kể đến giá cổ phiếu, mặc dù mức độ ảnh hưởng là rất nhỏ.
- Giá trị `p (P>|t|)`: Cả hai hệ số đều có giá trị p rất nhỏ (0.000), cho thấy chúng đều có ý nghĩa thống kê cao.
- Hệ số chặn (`Intercept`) là 77090. Đây là giá trị trung bình của giá cổ phiếu khi khối lượng giao dịch bằng 0.
- Hệ số của khối lượng giao dịch là -0.0039. Điều này có nghĩa là khi khối lượng giao dịch tăng lên 1 đơn vị (sau khi chuyển đổi), giá cổ phiếu giảm trung bình 0.0039 đơn vị.
- Phương trình hồi quy: `Price = 77090 - 0.0039 * Vol.`
- **Kết luận**: mặc dù khối lượng giao dịch có ý nghĩa thống kê trong việc ảnh hưởng đến giá cổ phiếu của Vietcombank, tỷ lệ giải thích của mô hình rất thấp (chỉ 4.2%). Điều này cho thấy có nhiều yếu tố khác ảnh hưởng đến giá cổ phiếu hơn là chỉ khối lượng giao dịch.
5. **TCB (Techcombank):**
- **Giá trị `R-squared` và `Adj. R-squared`**: 0.275, nghĩa là mô hình hồi quy giải thích được 27.5% biến động của giá cổ phiếu (Price) dựa trên khối lượng giao dịch (Vol.). Đây là tỷ lệ giải thích trung bình, cho thấy khối lượng giao dịch có một mức độ ảnh hưởng đáng kể đến giá cổ phiếu nhưng vẫn còn nhiều yếu tố khác cần được xem xét.
- **Giá trị `F-statistic`**: 562.8 với mức ý nghĩa (`Prob`) rất nhỏ (9.37e-106), cho thấy mô hình có ý nghĩa thống kê cao. Điều này nghĩa là khối lượng giao dịch có ảnh hưởng đáng kể đến giá cổ phiếu.
- **Giá trị `p (P>|t|)`**: Cả hai hệ số đều có giá trị p rất nhỏ (0.000), cho thấy chúng đều có ý nghĩa thống kê cao.
- **Hệ số chặn (`Intercept`)**: 27010. Đây là giá trị trung bình của giá cổ phiếu khi khối lượng giao dịch bằng 0.
- **Hệ số của khối lượng giao dịch**: 0.0008. Điều này có nghĩa là khi khối lượng giao dịch tăng lên 1 đơn vị (sau khi chuyển đổi), giá cổ phiếu tăng trung bình 0.0008 đơn vị.
- **Phương trình hồi quy**: `Price = 27010 + 0.0008 * Vol.`
- **Kết luận**: kết quả hồi quy cho thấy có một mối quan hệ tích cực giữa khối lượng giao dịch và giá cổ phiếu của Techcombank. Khối lượng giao dịch có thể giải thích được 27.5% biến động của giá cổ phiếu, và điều này có ý nghĩa thống kê cao. Tuy nhiên, vẫn còn 72.5% biến động của giá cổ phiếu được giải thích bởi các yếu tố khác không được bao gồm trong mô hình này.

### Câu hỏi: So sánh và nhận xét về giá cổ phiếu và khối lượng giao dịch theo thời gian giữa các ngân hàng. Nhận xét và lý giải về sự thay đổi thứ hạng nếu có